In [ ]:
key = 'your-api-key'

In [148]:
import requests
import numpy as np
import pandas as pd
import json
import base64
from sentence_transformers.util import cos_sim

/home/giri/ROE/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
url1="http://aiproxy.sanand.workers.dev/openai/v1/chat/completions"    # post
url2="http://aiproxy.sanand.workers.dev/openai/v1/embeddings"          # post
url3="https://aiproxy.sanand.workers.dev/openai/v1/models"             # get

# models

In [ ]:
m1="gpt-4o-mini"
m2="text-embedding-3-small"

In [7]:
header={
    "Content-Type" : "application/json",
    "Authorization" : f"Bearer {key}"
}

In [160]:
r1=requests.get(url3,headers=header)
d1=r1.json()

In [165]:
print(d1.keys() , d1['object'], sep="\n")

dict_keys(['object', 'data'])
list


In [172]:
print(type(d1['data']))
d1['data'][0]

<class 'list'>


{'id': 'tts-1',
 'object': 'model',
 'created': 1681940951,
 'owned_by': 'openai-internal'}

# sentiment analysis

In [ ]:
data_2 = {
    "model" : m1,
    "messages": [
        {
            "role": "system",
            "content": "Classify the sentiment of the following three movie reviews. Respond with only one word per review: positive, negative, or neutral."
        },
        {
            "role": "user",
            "content": 
                "1. The cinematography was breathtaking, and the performances were outstanding.\n"
                "2. The plot was confusing, and the dialogues were poorly written.\n"
                "3. The movie had both entertaining and boring moments in equal measure."
        }
    ]
}

In [ ]:
r2=requests.post(url1,headers=header,data=json.dumps(data_2))
d2=r2.json()

In [ ]:
d2['choices']

{'id': 'chatcmpl-B6M3y7HPpVH8KVHyu6wPDfNa3ut8h',
 'object': 'chat.completion',
 'created': 1740854478,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '1. positive  \n2. negative  \n3. neutral  ',
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 79,
  'completion_tokens': 13,
  'total_tokens': 92,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': 'fp_06737a9306',
 'monthlyCost': 0.000315,
 'cost': 0.000315,
 'monthlyRequests': 1}

### sentiment analysis with json schema

pls look at structured.json

In [33]:
data_3 = {
    "model" : m1,
    "messages": [
        {
            "role": "system",
            "content": "Classify the sentiment of the following three movie reviews. Respond with only one word per review: positive, negative, or neutral."
        },
        {
            "role": "user",
            "content":
                "0. I hate using LLMs through API calls \n"  
                "1. The cinematography was breathtaking, and the performances were outstanding.\n"
                "2. The plot was confusing, and the dialogues were poorly written.\n"
                "3. The movie had both entertaining and boring moments in equal measure."
        }
    ],
    "response_format": {
        "type": "json_schema",
        "json_schema": {
            "name": "sentiment_analysis",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                    "resp": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "sentence_no": {
                                    "type": "integer",
                                    "description": "This should be the refence sentence number, which will be given in the prompt. If not given, assign a number from the context and sentence position"
                                },
                                "sentiment": {
                                    "type": "string"
                                }
                            },
                            "required": [
                                "sentence_no",
                                "sentiment"
                            ],
                            "additionalProperties": False
                        }
                    }
                },
                "required": [
                    "resp"
                ],
                "additionalProperties": False
            }
        }
    }
}

In [34]:
r3 = requests.post(url1,headers=header,data=json.dumps(data_3))
d3=r3.json()

In [ ]:
json.d3

{'id': 'chatcmpl-B6MqfdQEJXX56OV3gZhmbFEg8xkHF',
 'object': 'chat.completion',
 'created': 1740857497,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '{"resp":[{"sentence_no":0,"sentiment":"negative"},{"sentence_no":1,"sentiment":"positive"},{"sentence_no":2,"sentiment":"negative"},{"sentence_no":3,"sentiment":"neutral"}]}',
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 175,
  'completion_tokens': 46,
  'total_tokens': 221,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': 'fp_06737a9306',
 'monthlyCost': 0.003309,
 'cost': 0.000801,
 'monthlyRequests': 6}

In [39]:
print(json.loads(d3['choices'][0]['message']['content']))

{'resp': [{'sentence_no': 0, 'sentiment': 'negative'}, {'sentence_no': 1, 'sentiment': 'positive'}, {'sentence_no': 2, 'sentiment': 'negative'}, {'sentence_no': 3, 'sentiment': 'neutral'}]}


# embeddings and cosine similarity

In [ ]:
words = ['Apple', 'Orange', 'Banana', 'Jamaica', 'Sri Lanka', 'Facebook', 'Google']

data_4 = {
    "input": words,
    "model": m2,
    "encoding_format": "float"
}

In [49]:
r4 = requests.post(url2,headers=header,data=json.dumps(data_4))

d4=r4.json()

In [146]:
embed=[]
for i,j in zip(words,d4['data']):
    embed.append({'word' : i , 'embedding' : j['embedding']})
df = pd.DataFrame(embed)
df

,word,embedding
0,Apple,"[0.009175114, -0.03518077, -0.025036855, 0.039..."
1,Orange,"[-0.013172907, 0.0051653804, -0.023958141, 0.0..."
2,Banana,"[0.021114137, -0.043225136, -0.035277914, 0.01..."
3,Jamaica,"[0.034378856, -0.031084437, 0.006070764, 0.051..."
4,Sri Lanka,"[0.02704939, -0.016649535, 0.037278358, -0.004..."
5,Facebook,"[0.030002322, -0.025417516, -0.045874864, 0.09..."
6,Google,"[0.009119771, 0.0033862204, -0.017700443, 0.03..."


In [158]:
embed_matrix = pd.DataFrame(
    cos_sim(df.embedding,df.embedding).numpy(),
    index=list(df.word),
    columns=list(df.word)
)

embed_matrix

,Apple,Orange,Banana,Jamaica,Sri Lanka,Facebook,Google
Apple,1.000000,0.445330,0.386284,0.207051,0.195969,0.409232,0.437972
Orange,0.445330,1.000000,0.357203,0.198378,0.137683,0.297718,0.257831
Banana,0.386284,0.357203,1.000000,0.367952,0.238129,0.209775,0.218002
Jamaica,0.207051,0.198378,0.367952,1.000000,0.395514,0.185169,0.124608
Sri Lanka,0.195969,0.137683,0.238129,0.395514,1.000000,0.182873,0.182547
Facebook,0.409232,0.297718,0.209775,0.185169,0.182873,1.000000,0.582012
Google,0.437972,0.257831,0.218002,0.124608,0.182547,0.582012,1.000000


# image analysis

one with image in base 64 uri. pls look at base64.ipynb

another with wikipedia image with `"details" : "low"`

In [ ]:
with open('email.png', 'rb') as f:
    binary_data = f.read()
    image_b64 = base64.b64encode(binary_data).decode()

uri=f"data:image/png;base64,{image_b64}"

In [102]:
data_5 = {
    "model" : m1,
    "messages" : [
        {
            "role" : "user",
            "content" : [
        {"type": "text", "text": "What is in this image?"},
        {
          "type": "image_url",
          "image_url": { "url": uri}
        }
      ]
        }
    ]
}

In [103]:
r5 = requests.post(url1,headers=header,data=json.dumps(data_5))

d5 = r5.json()

In [104]:
d5['choices'][0]['message']['content']

'The image contains an email address and a number. The email address is "23f2000573@ds.study.iitm.ac.in" and the number is "22005475."'

In [90]:
data_6 = {
    "model": m1,
    "messages": [
      {
        "role": "user",
        "content": [
          {"type": "text", "text": "What is in this image?"},
          {
            "type": "image_url",
            "image_url": {"url": "https://upload.wikimedia.org/wikipedia/commons/3/34/Correlation_coefficient.png","detail": "low"}
          }
        ]
      }
    ]
}

In [91]:
r6 = requests.post(url1,headers=header,data=json.dumps(data_6))

d6 = r6.json()

In [94]:
print(d6['choices'][0]['message']['content'])

The image displays several scatter plots illustrating different correlation coefficients (\( \rho \)). Each plot shows a set of orange points and a green line representing the relationship between the variables:

1. **\( \rho = -1 \)** - Perfect negative correlation; points lie on a straight line with a downward slope.
2. **\( -1 < \rho < 0 \)** - Negative correlation; points are scattered but still show a downward trend.
3. **\( 0 < \rho < 1 \)** - Positive correlation; points are scattered but indicate an upward trend.
4. **\( \rho = 1 \)** - Perfect positive correlation; points lie on a straight line with an upward slope.
5. **\( \rho = 0 \)** - No correlation; points are randomly scattered with no discernible trend.

These plots visually represent varying degrees of linear relationships between two variables.


# function calling

In [119]:
def string_repeat(s, n):
  return s * n

def string_slice(s, n):
  if abs(n) >= len(s):
    return s
  return s[:abs(n)]

func_dict = {
  'string_repeat' : string_repeat,
  "string_slice" : string_slice
}

In [110]:
function_tools = [
    
    {
        "type": "function",
        "function": {
            "name": "string_repeat",
            "description" : "a python function which takes a string 's' and number 'n' as arguments and returns the string 's*n'",
            "parameters": {
                "type": "object",
                "properties": {
                    "s" : {
                        "type" : "string",
                        "description" : "the string which will be multiplied by 'n'"
                    },
                    "n" : {
                        "type" : "integer",
                        "description" : "the number which decides the number of times string 's' is repeated"
                    }
                },
                "required": ['s','n'],
                "additionalProperties": False
            },
            "strict": True
        }
    },

    {
        "type": "function",
        "function": {
            "name": "string_slice",
            "description" : "a python function which takes a string 's' and number 'n' as arguments and returns the slice 's[:abs(n)]'. this is the first 'n' characters of 's'",
            "parameters": {
            "type": "object",
                "properties": {
                    "s" : {
                        "type" : "string",
                        "description" : "the string which will be sliced"
                    },
                    "n" : {
                        "type" : "integer",
                        "description" : "the number which decides the number of charecters to be slice in string 's'"
                    }
                },
                "required": ['s','n'],
                "additionalProperties": False
            },
            "strict": True
        }
    }

]

In [140]:
def query(user_input,url,tools,header)   : 
    data_7 = {
        "model" : m1,
        "messages" :[{"role": "user", "content": user_input}],
        "tools" : tools,
        "tool_choice" : "auto"
    }

    r = requests.post(url,headers=header,data=json.dumps(data_7))

    return r.json()

In [116]:
d7 = query("is it possible to print appleappleapple",url1,function_tools,header)

In [ ]:
d7

{'id': 'chatcmpl-B6PBfTx14nsC5CATjk4mAeSJlGzyC',
 'object': 'chat.completion',
 'created': 1740866487,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'tool_calls': [{'id': 'call_iK13ZeOriwJkJmBe3zRGKPJt',
      'type': 'function',
      'function': {'name': 'string_repeat',
       'arguments': '{"s":"apple","n":3}'}}],
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'tool_calls'}],
 'usage': {'prompt_tokens': 185,
  'completion_tokens': 19,
  'total_tokens': 204,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': 'fp_06737a9306',
 'monthlyCost': 0.01940424,
 'cost': 0.0006690000000000001,
 'monthlyRequests': 15}

In [ ]:
tool_1 = d7['choices'][0]['message']['tool_calls'][0]

print(tool_1)


# not sure why, but the 'arguments' key's value is not a dict, but a string, hence used json.loads

func_name_1 = tool_1['function']['name']
str_1 = json.loads(tool_1['function']['arguments'])['s']
int_1 = json.loads(tool_1['function']['arguments'])['n']
print(func_name_1,str_1,int_1)

print(func_dict[func_name_1](s=str_1,n=int_1))

{'id': 'call_iK13ZeOriwJkJmBe3zRGKPJt', 'type': 'function', 'function': {'name': 'string_repeat', 'arguments': '{"s":"apple","n":3}'}}
string_repeat apple 3
appleappleapple


In [141]:
d8 = query("is it possible to print take the first 3 elements of abcdefghij",url1,function_tools,header)

In [142]:
d8

{'id': 'chatcmpl-B6PNmzPH4J04KfgZJAPZjXshvmW6W',
 'object': 'chat.completion',
 'created': 1740867238,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'tool_calls': [{'id': 'call_bgb1s1UYo68eBdgcHQXzu1W8',
      'type': 'function',
      'function': {'name': 'string_slice',
       'arguments': '{"s":"abcdefghij","n":3}'}}],
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'tool_calls'}],
 'usage': {'prompt_tokens': 193,
  'completion_tokens': 20,
  'total_tokens': 213,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': 'fp_06737a9306',
 'monthlyCost': 0.02080224,
 'cost': 0.000699,
 'monthlyRequests': 17}

In [ ]:
tool_2 = d8['choices'][0]['message']['tool_calls'][0]

print(tool_2)

# not sure why, but the 'arguments' key's value is not a dict, but a string, hence used json.loads

func_name_2 = tool_2['function']['name']
str_2 = json.loads(tool_2['function']['arguments'])['s']
int_2 = json.loads(tool_2['function']['arguments'])['n']
print(func_name_2,str_2,int_2)

print(func_dict[func_name_2](s=str_2,n=int_2))

{'id': 'call_bgb1s1UYo68eBdgcHQXzu1W8', 'type': 'function', 'function': {'name': 'string_slice', 'arguments': '{"s":"abcdefghij","n":3}'}}
string_slice abcdefghij 3
abc
